# Find CA List for 20 Clinics
Query CAs from role_assignments collection for specified organizations

In [1]:
from pymongo import MongoClient
from bson import ObjectId
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Connect to MongoDB
mongo_uri = os.getenv('MONGO_DATABASE_URI')
client = MongoClient(mongo_uri)
db = client.get_default_database()

print(f"Connected to database: {db.name}")

Connected to database: UnifiedCare


In [3]:
# Define the 20 organization IDs (clinics)
organization_ids = [
    # Add your 20 organization ObjectIds here
    # Example:
    ObjectId("5f4590bbe8f9890013dc3e26"),
    # ObjectId("..."),
]

print(f"Looking for CAs in {len(organization_ids)} organizations")

Looking for CAs in 1 organizations


In [4]:
# Query role_assignments for active CAs in these organizations
query = {
    'active': True,
    'roleType': 'CA',
    'organizationId': {'$in': organization_ids}
}

role_assignments = list(db.role_assignments.find(query))
print(f"Found {len(role_assignments)} active CA role assignments")

Found 0 active CA role assignments


In [5]:
# Extract member IDs and organize by organization
ca_data = []

for assignment in role_assignments:
    ca_data.append({
        'memberId': str(assignment['memberId']),
        'organizationId': str(assignment['organizationId']),
        'roleType': assignment['roleType'],
        'active': assignment['active']
    })

df_cas = pd.DataFrame(ca_data)
print(f"\nTotal CA assignments: {len(df_cas)}")
df_cas.head(10)


Total CA assignments: 0


""


In [6]:
# Get unique member IDs (all CAs)
unique_member_ids = df_cas['memberId'].unique()
print(f"\nUnique CAs (member IDs): {len(unique_member_ids)}")
print(f"\nAll member IDs:")
for mid in unique_member_ids:
    print(f"  ObjectId('{mid}')")

KeyError: 'memberId'

In [ ]:
# Get CA details from members collection
member_ids_obj = [ObjectId(mid) for mid in unique_member_ids]
ca_members = list(db.members.find({'_id': {'$in': member_ids_obj}}))

ca_details = []
for member in ca_members:
    ca_details.append({
        'memberId': str(member['_id']),
        'firstName': member.get('firstName', ''),
        'lastName': member.get('lastName', ''),
        'email': member.get('email', ''),
        'fullName': f"{member.get('firstName', '')} {member.get('lastName', '')}"
    })

df_ca_details = pd.DataFrame(ca_details)
print(f"\nCA Details found: {len(df_ca_details)}")
df_ca_details

In [ ]:
# Merge CA details with assignments
df_full = df_cas.merge(df_ca_details, on='memberId', how='left')
df_full = df_full.sort_values(['organizationId', 'fullName'])
print(f"\nFull CA list with organization mapping:")
df_full

In [ ]:
# Summary by organization
print("\n=== CAs per Organization ===")
org_summary = df_full.groupby('organizationId').agg({
    'memberId': 'count',
    'fullName': lambda x: ', '.join(x.unique())
}).rename(columns={'memberId': 'CA_count', 'fullName': 'CAs'})
org_summary

In [ ]:
# Export results
df_full.to_csv('ca_list_all.csv', index=False)
df_ca_details.to_csv('ca_unique_members.csv', index=False)
org_summary.to_csv('ca_by_organization.csv')

# Save member IDs to text file
with open('ca_member_ids.txt', 'w') as f:
    for mid in unique_member_ids:
        f.write(f"ObjectId('{mid}')\n")

print("\nExported files:")
print("  - ca_list_all.csv: All CA assignments with details")
print("  - ca_unique_members.csv: Unique CA members")
print("  - ca_by_organization.csv: Summary by organization")
print("  - ca_member_ids.txt: List of member ObjectIds")

In [ ]:
# Display the list in a readable format
print("\n" + "="*80)
print("CA LIST BY ORGANIZATION")
print("="*80)

for org_id in df_full['organizationId'].unique():
    org_cas = df_full[df_full['organizationId'] == org_id]
    print(f"\nOrganization: {org_id}")
    print(f"Total CAs: {len(org_cas)}")
    for _, ca in org_cas.iterrows():
        print(f"  - {ca['fullName']} (ID: {ca['memberId']})")
    print("-" * 80)